In [ ]:
from pathlib import Path

import plotly.graph_objects as go
import polars as pl

from plotly.subplots import make_subplots

from afft.io import read_file
from afft.utils.log import logger

In [ ]:
from afft.services.sirius_messages import (
    EcopuckMessage,
)

from afft.services.sirius_messages import (
    parse_message_header,
    parse_ecopuck_message,
)

### Load data

In [ ]:
paths: dict = dict()

paths["datadir"] = Path(
    "/media/martin/pcie_01/acfr_revisits_processed/acfr_merged_message_files"
)
paths["messages"] = paths["datadir"] / Path("qdc5ghs3_20130405_103429_messages.txt")

lines: list[str] = read_file(paths["messages"]).unwrap()


TOPIC_TO_PARSER: dict = {
    "ECOPUCK": parse_ecopuck_message,
}


def parse_messages(lines: list[str], parsers: dict) -> dict:
    """TODO"""
    parsed_messages: dict[type, object] = dict()

    for line in lines:
        header = parse_message_header(line).unwrap()

        if header.topic not in parsers:
            continue

        parsed: object = parsers[header.topic](line).unwrap()

        if type(parsed) not in parsed_messages:
            parsed_messages[type(parsed)] = list()
        else:
            parsed_messages[type(parsed)].append(parsed)

    return parsed_messages


parsed_messages: dict[type, list[object]] = parse_messages(lines, TOPIC_TO_PARSER)

for message_type, messages in parsed_messages.items():
    logger.info(f"{message_type}: {len(messages)}")

### Define plot functions

In [ ]:
def plot_ecopuck_series(data: pl.DataFrame) -> None:
    """TODO"""
    fig = make_subplots(
        rows=4,
        cols=1,
        subplot_titles=("Chlorophyll", "Backscatter", "cDOM", "Temperature"),
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["chlorophyll"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["backscatter"],
        ),
        row=2,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["cdom"],
        ),
        row=3,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["temperature"],
        ),
        row=4,
        col=1,
    )

    fig.update_layout(height=800, width=1000, title_text="LQ Modem Timeseries")

    return fig

### Format LQ Modem messages

In [ ]:
data: dict = dict()
data["ecopuck"]: pl.DataFrame = pl.DataFrame(
    [
        {
            "topic": message.header.topic,
            "timestamp": message.header.timestamp,
            "chlorophyll": message.body.chlorophyll,
            "backscatter": message.body.backscatter,
            "cdom": message.body.cdom,
            "temperature": message.body.temperature,
        }
        for message in parsed_messages[EcopuckMessage]
    ]
)

figs: dict = dict()
figs["ecopuck"] = plot_ecopuck_series(data["ecopuck"])
figs["ecopuck"].show()